In [1]:
import json
import numpy as np
import pandas as pd
import multiprocessing

from pymoo.optimize import minimize
from pymoo.algorithms.moo.rvea import RVEA
from pymoo.core.problem import ElementwiseProblem, StarmapParallelization
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.pntx import SinglePointCrossover

In [2]:
# Read CSVs
df_classrooms = pd.read_csv("./datasets/clean/CaracterizaçãoDasSalasSmall.csv", sep=";", header=0)
df_classes = pd.read_csv("./datasets/clean/HorarioDeExemploSmall.csv", sep=";", header=0)

def get_characteristics(row):
  cols_with_x = [col for col in row.index if row[col] == "X"]
  return ", ".join(cols_with_x)

# Prepare df_classrooms
df_classrooms["caracteristicas"] = df_classrooms.apply(get_characteristics, axis=1)
df_classrooms = df_classrooms[["Nome sala", "Capacidade Normal", "caracteristicas"]]
df_classrooms = df_classrooms.rename(columns={"Nome sala":"Sala da aula", "Capacidade Normal":"Lotação", "caracteristicas":"Características reais da sala"})
print("df_classrooms len: ", len(df_classrooms))

# Prepare df_classes
df_classes["Sala da aula"] = ""
df_classes["Lotação"] = ""
df_classes["Características reais da sala"] = ""
print("df_classes len: ", len(df_classes))

df_classrooms len:  85
df_classes len:  5746


In [3]:
class ISCTE_CLASSES(ElementwiseProblem):
  def __init__(self, df_classes, df_classrooms, **kwargs):
    self.df_classes = df_classes
    self.df_classrooms = df_classrooms

    n_var = len(df_classes)

    xl = np.zeros(n_var)
    xu = np.array([len(df_classrooms) - 1] * len(df_classes))

    super().__init__(n_var=n_var, n_obj=3, xl=xl, xu=xu, **kwargs)


  def _evaluate(self, x, out, *args, **kwargs):
    df_schedule = self.arr_to_df(x)
    schedule_eval = self.df_evaluate(df_schedule)

    out["F"] = np.array(schedule_eval)


  def arr_to_df(self, arr_schedule):
    df_schedule = self.df_classes.copy()

    for i, row in df_schedule.iterrows():
      if arr_schedule[i] >= 97:
        print(arr_schedule)

      classroom = self.df_classrooms.iloc[int(arr_schedule[i])]

      df_schedule.at[i, "Sala da aula"] = classroom.iloc[0]
      df_schedule.at[i, "Lotação"] = classroom.iloc[1]
      df_schedule.at[i, "Características reais da sala"] = classroom.iloc[2]

    df_schedule["Datetime Início"] = pd.to_datetime(df_schedule['Dia'] + ' ' + df_schedule['Início'], format='%d/%m/%Y %H:%M:%S')
    df_schedule["Datetime Fim"] = pd.to_datetime(df_schedule['Dia'] + ' ' + df_schedule['Fim'], format='%d/%m/%Y %H:%M:%S')
    df_schedule["Lotação - Inscritos no turno"] = df_schedule.apply(lambda x: int(x["Lotação"]) - int(x["Inscritos no turno"]), axis=1)
    
    return df_schedule


  def check_overlap(self, start, end, df):
    for _, row in df.iterrows():
      tmp_start = row["Datetime Início"]
      tmp_end = row["Datetime Fim"]

      if tmp_start <= start < tmp_end or tmp_start < end <= tmp_end:
        return True
      
    return False


  def df_evaluate(self, df):
    sobreposicao = 0
    superlotacao = (df["Lotação - Inscritos no turno"] < 0).sum()
    caracteristicas = df.apply(lambda row: row["Características da sala pedida para a aula"] not in row["Características reais da sala"], axis=1).sum()

    for i, row in df.iterrows():
      tmp_df = df.loc[i + 1:].copy()
      tmp_df = tmp_df[(tmp_df['Sala da aula'] == row['Sala da aula']) & (tmp_df['Dia'] == row['Dia'])].copy()
      
      if tmp_df.shape[0] > 0:
        if self.check_overlap(row["Datetime Início"], row["Datetime Fim"], tmp_df):
          sobreposicao += 1

    return [sobreposicao, superlotacao, caracteristicas]

In [4]:
# the number of process to be used
n_process = 6

# initialize the pool
pool = multiprocessing.Pool(n_process)

runner = StarmapParallelization(pool.starmap)

iscte_problem = ISCTE_CLASSES(df_classes, df_classrooms, elementwise_runner=runner)

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=13)

rvea = RVEA(
  pop_size=105,
  ref_dirs=ref_dirs,
  sampling=IntegerRandomSampling(),
  crossover=SinglePointCrossover(prob=1),
  mutation=PolynomialMutation(prob=1, eta=10, repair=RoundingRepair()),
  eliminate_duplicates=True
)

iscte_res = minimize(
  iscte_problem,
  rvea,
  seed=7,
  save_history=True,
  verbose=True,
  termination=('n_eval', 10500)
)

print('Threads:', iscte_res.exec_time)
# pool.close()

X = iscte_res.opt.get("X")
F = iscte_res.opt.get("F")

# print(X)
print(F)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      105 |    105 |             - |             -
     2 |      210 |     40 |  0.0625000000 |         ideal
     3 |      315 |     43 |  0.1976744186 |         ideal
     4 |      420 |     47 |  0.1014492754 |         ideal
     5 |      525 |     49 |  0.0875912409 |         ideal
     6 |      630 |     45 |  0.1564625850 |         ideal
     7 |      735 |     49 |  0.0549450549 |         ideal
     8 |      840 |     49 |  0.0691244240 |         ideal
     9 |      945 |     51 |  0.0529801325 |         ideal
    10 |     1050 |     46 |  0.0503144654 |         ideal
    11 |     1155 |     44 |  0.0503144654 |         ideal
    12 |     1260 |     53 |  0.0657894737 |         ideal
    13 |     1365 |     50 |  0.0454545455 |         ideal
    14 |     1470 |     59 |  0.1014492754 |         nadir
    15 |     1575 |     53 |  0.0782122905 |         ideal
    16 |     1680 |     55 |  0.0271739130 |         ide

In [5]:
pool.close()

In [6]:
print(iscte_res.history[0].opt.get("F"))

[[ 482. 1309. 2505.]
 [ 480. 1311. 2496.]
 [ 534. 1306. 2493.]
 [ 495. 1305. 2463.]
 [ 484. 1357. 2454.]
 [ 504. 1341. 2499.]
 [ 489. 1340. 2457.]
 [ 481. 1273. 2445.]
 [ 474. 1303. 2434.]
 [ 514. 1309. 2467.]
 [ 488. 1334. 2443.]
 [ 506. 1264. 2468.]
 [ 542. 1308. 2495.]
 [ 501. 1314. 2437.]
 [ 511. 1284. 2463.]
 [ 496. 1240. 2483.]
 [ 507. 1359. 2504.]
 [ 477. 1307. 2469.]
 [ 498. 1282. 2517.]
 [ 534. 1345. 2530.]
 [ 486. 1311. 2457.]
 [ 498. 1358. 2468.]
 [ 502. 1310. 2528.]
 [ 505. 1323. 2462.]
 [ 506. 1260. 2488.]
 [ 517. 1311. 2405.]
 [ 481. 1268. 2478.]
 [ 494. 1308. 2471.]
 [ 512. 1291. 2467.]
 [ 515. 1326. 2487.]
 [ 520. 1310. 2483.]
 [ 460. 1326. 2503.]
 [ 519. 1330. 2488.]
 [ 482. 1301. 2488.]
 [ 476. 1283. 2397.]
 [ 470. 1288. 2498.]
 [ 499. 1278. 2493.]
 [ 504. 1262. 2518.]
 [ 479. 1336. 2464.]
 [ 526. 1276. 2501.]
 [ 482. 1311. 2480.]
 [ 506. 1302. 2470.]
 [ 502. 1313. 2485.]
 [ 503. 1326. 2472.]
 [ 499. 1267. 2493.]
 [ 529. 1298. 2485.]
 [ 462. 1325. 2518.]
 [ 500. 1310.

In [8]:
for i in [0, 9, 24, 49, 74, 98]:
  score_json = json.dumps(iscte_res.history[i].opt.get("F").tolist(), indent=4)
  schedule_json = json.dumps(iscte_res.history[i].opt.get("X").tolist(), indent=4)

  with open(f"./results/final/RVEA/score_gen_{i+1}.json", "w") as outfile:
    outfile.write(score_json)
    
  with open(f"./results/final/RVEA/schedule_gen_{i+1}.json", "w") as outfile:
    outfile.write(schedule_json)

In [ ]:
# result_dict = {"Score": F.tolist(), "Schedule": X.tolist()}

# results_json = json.dumps(result_dict, indent=4)

# with open("./results/RVEA_result_2_1.json", "w") as outfile:
#   outfile.write(results_json)